# Lezione 4 — Analisi Meteo: GroupBy e Transform

---

## Obiettivi della Lezione

Al termine di questa lezione sarai in grado di:

1. Usare **quantili** per definire soglie basate sui dati
2. Creare **colonne booleane** per classificazioni binarie
3. Categorizzare variabili continue in **classi discrete**
4. Applicare **`.groupby()`** per aggregazioni multi-colonna
5. Usare **`.transform()`** per aggiungere statistiche di gruppo a ogni riga
6. Confrontare valori individuali con medie di gruppo

---

## Prerequisiti

- Lezione 2: Filtri e Feature Engineering con Pandas
- Conoscenza di boolean indexing e `.loc[]`

---

## Indice

1. **SEZIONE 1** — Teoria Concettuale Approfondita
2. **SEZIONE 2** — Schema Mentale / Mappa Decisionale
3. **SEZIONE 3** — Notebook Dimostrativo
4. **SEZIONE 4** — Metodi Spiegati
5. **SEZIONE 5** — Glossario
6. **SEZIONE 6** — Errori Comuni e Debug Rapido
7. **SEZIONE 7** — Conclusione Operativa
8. **SEZIONE 8** — Checklist di Fine Lezione
9. **SEZIONE 9** — Changelog Didattico

---

## Librerie Utilizzate

```python
import pandas as pd
import numpy as np
```

---

# SEZIONE 1 — Teoria Concettuale Approfondita

## 1.1 Quantili e Percentili

I **quantili** dividono i dati ordinati in parti uguali:

| Quantile | Percentile | Significato |
|----------|------------|-------------|
| Q1 | 25° | Il 25% dei dati sta sotto questo valore |
| Q2 (Mediana) | 50° | Il 50% dei dati sta sotto |
| Q3 | 75° | Il 75% dei dati sta sotto |

**Formula per il percentile p:**
$$Q_p = \text{valore tale che } p\% \text{ dei dati } \leq Q_p$$

**Uso pratico:** Definire soglie "alte" o "basse" basate sui dati stessi, non su valori arbitrari.

## 1.2 Categorizzazione di Variabili Continue

Trasformare numeri in categorie permette:
- Analisi piu intuitive ("vento alto" vs "vento 17.3 km/h")
- Raggruppamenti significativi
- Visualizzazioni piu chiare

**Approcci comuni:**
1. **Soglie fisse:** definite da conoscenza del dominio
2. **Soglie basate sui dati:** quantili, media +/- std
3. **`pd.cut()`:** binning automatico

## 1.3 GroupBy: Split-Apply-Combine

Il paradigma **Split-Apply-Combine** e il cuore di `.groupby()`:

```
DATI ORIGINALI
     |
     v
  SPLIT: dividi per gruppi
     |
     v
  APPLY: calcola statistica per gruppo
     |
     v
  COMBINE: unisci i risultati
```

**Esempio:**
```python
df.groupby("Categoria")["Valore"].mean()
```
1. Split: separa righe per ogni Categoria
2. Apply: calcola media di Valore per gruppo
3. Combine: restituisce Series con medie

## 1.4 La Differenza Cruciale: `agg()` vs `transform()`

| Metodo | Output | Righe | Uso |
|--------|--------|-------|-----|
| `.agg()` / `.mean()` | Una riga per gruppo | Ridotto | Statistiche riassuntive |
| `.transform()` | Stesse righe dell'originale | Invariato | Aggiungere statistica a ogni riga |

**Visualizzazione:**
```
Originale (7 righe)     agg('mean')        transform('mean')
   A    Val              A    mean           A    Val   mean_A
   X    10               X    15             X    10    15
   X    20               Y    25             X    20    15
   Y    30                                   Y    30    25
   Y    20                                   Y    20    25
```

## 1.5 Perche Usare Transform?

`transform()` e fondamentale quando vuoi:
- Confrontare ogni riga con la media del suo gruppo
- Calcolare z-score per gruppo
- Creare feature "normalizzate per gruppo"
- Mantenere la struttura originale del DataFrame

**Pattern comune:**
```python
# Quanto sopra/sotto la media del gruppo?
df["Diff_da_Media"] = df["Valore"] - df.groupby("Gruppo")["Valore"].transform("mean")
```

## 1.6 GroupBy Multi-Colonna

Si possono creare gruppi basati su combinazioni di colonne:

```python
df.groupby(["Pioggia", "Vento_Class"])["Umidita"].mean()
```

Questo crea gruppi per ogni combinazione unica:
- (Pioggia=0, Vento=Basso)
- (Pioggia=0, Vento=Medio)
- (Pioggia=1, Vento=Alto)
- ecc.

In [ ]:
# === SETUP: Import librerie e creazione DataFrame ===

import pandas as pd   # Libreria per manipolazione dati tabulari
import numpy as np    # Libreria per operazioni numeriche

# Creiamo un DataFrame con dati meteorologici settimanali
df = pd.DataFrame({
    "Giorno": ["Lun", "Mar", "Mer", "Gio", "Ven", "Sab", "Dom"],
    "Temperatura": [18, 20, 21, 19, 23, 25, 22],  # Gradi Celsius
    "Umidita": [60, 55, 58, 63, 50, 45, 52],      # Percentuale
    "Pioggia": [0, 1, 0, 1, 0, 0, 1],             # 0=No, 1=Si
    "Vento": [5, 12, 8, 20, 7, 6, 15]             # km/h
})

# Visualizziamo il DataFrame
print("DataFrame meteo creato con successo!")
print(f"Dimensioni: {df.shape[0]} righe x {df.shape[1]} colonne")
print(f"Colonne: {list(df.columns)}")
print()
df

,Giorno,Temperatura,Umidita,Pioggia,Vento
0,Lun,18,60,0,5
1,Mar,20,55,1,12
2,Mer,21,58,0,8
3,Gio,19,63,1,20
4,Ven,23,50,0,7
5,Sab,25,45,0,6
6,Dom,22,52,1,15


---

# SEZIONE 2 — Schema Mentale / Mappa Decisionale

## Quando Usare Quale Tecnica?

```
DEVO ANALIZZARE DATI PER GRUPPI
                |
                v
    Voglio statistiche RIASSUNTIVE per gruppo?
           /            \
         SI              NO
          |               |
          v               v
   .groupby().agg()   Voglio aggiungere stat
   .groupby().mean()  a OGNI RIGA?
          |                    |
          v                    v
   Output ridotto        .transform()
   (1 riga/gruppo)      Output stesso size
```

## Scelta della Funzione di Aggregazione

```
CHE STATISTICA MI SERVE?
    /    |    |    \
 MEDIA  SOMMA  CONTA  CUSTOM
   |      |      |       |
.mean() .sum() .count() .agg(func)
```

## Pattern di Categorizzazione

```
VARIABILE CONTINUA → CATEGORICA
           |
           v
    Conosco le soglie?
       /        \
     SI          NO
      |           |
      v           v
  .loc[] o     pd.cut() o
  np.where()  pd.qcut()
```

## Checklist Pre-GroupBy

1. Quali colonne definiscono i gruppi?
2. Quale colonna voglio aggregare?
3. Quale funzione di aggregazione?
4. Mi serve output ridotto (`agg`) o espanso (`transform`)?

---

# SEZIONE 3 — Notebook Dimostrativo

## 3.1 Setup: Import e Creazione DataFrame

**Perche questo passaggio:** Creiamo un dataset meteo settimanale con 5 variabili. Questo ci permettera di esplorare categorizzazioni, groupby e transform.

---

## 3.2 Calcolo Quantili per Soglie Dinamiche

**Perche questo passaggio:** Invece di usare una soglia arbitraria (es. 60%), calcoliamo il 75° percentile (Q3) dell'umidita. Cosi la soglia si adatta automaticamente ai dati.

**Concetto chiave:** `.quantile(0.75)` restituisce il valore sotto cui sta il 75% dei dati.

In [ ]:
# === CALCOLO Q3 (75° percentile) DELL'UMIDITA ===

# Il 75° percentile e il valore sotto cui sta il 75% dei dati
q3_umidita = df["Umidita"].quantile(0.75)

print(f"75° percentile (Q3) dell'umidita: {q3_umidita}%")
print(f"Interpretazione: il 75% dei giorni ha umidita <= {q3_umidita}%")
print(f"I giorni con umidita > {q3_umidita}% sono nel top 25% piu umido")

# --- MICRO-CHECKPOINT ---
# Q3 deve essere tra min e max
assert df["Umidita"].min() <= q3_umidita <= df["Umidita"].max(), "Q3 fuori range!"
print(f"\n--- Micro-checkpoint: Q3 = {q3_umidita} (range: {df['Umidita'].min()}-{df['Umidita'].max()}) ---")

np.float64(59.0)

---

## 3.3 Creare una Colonna Booleana (Flag)

**Perche questo passaggio:** Creiamo una colonna `umido_alto` che vale True per i giorni con umidita sopra Q3. Questo semplifica filtri e analisi successive.

**Pattern:** `df["flag"] = df["colonna"] > soglia`

In [ ]:
# === CREAZIONE COLONNA BOOLEANA ===

# Creiamo un flag True/False per "umidita alta" (sopra Q3)
df["umido_alto"] = df["Umidita"] > q3_umidita

# --- MICRO-CHECKPOINT ---
n_umidi_alti = df["umido_alto"].sum()
print(f"Giorni con umidita > Q3 ({q3_umidita}%): {n_umidi_alti}")
print(f"Percentuale: {n_umidi_alti / len(df) * 100:.1f}%")

# Per definizione, circa 25% dei dati dovrebbe essere sopra Q3
assert n_umidi_alti <= len(df), "Conteggio non valido!"
print(f"\n--- Micro-checkpoint: {n_umidi_alti} giorni classificati come 'umido_alto' ---")

df

,Giorno,Temperatura,Umidita,Pioggia,Vento,umido_alto
0,Lun,18,60,0,5,True
1,Mar,20,55,1,12,False
2,Mer,21,58,0,8,False
3,Gio,19,63,1,20,True
4,Ven,23,50,0,7,False
5,Sab,25,45,0,6,False
6,Dom,22,52,1,15,False


---

## 3.4 Categorizzare una Variabile Continua

**Perche questo passaggio:** Trasformiamo la velocita del vento (numero continuo) in categorie discrete: Basso, Medio, Alto. Questo facilita l'analisi per gruppi.

**Soglie scelte:**
- **Basso:** Vento < 8 km/h
- **Medio:** 8 <= Vento < 15 km/h
- **Alto:** Vento >= 15 km/h

In [ ]:
# === STEP 1: Creare le maschere booleane per ogni categoria ===

# Definiamo le condizioni per ogni classe di vento
vento_basso = df["Vento"] < 8
vento_medio = (df["Vento"] >= 8) & (df["Vento"] < 15)
vento_alto = df["Vento"] >= 15

# Verifichiamo le maschere
print("Maschere booleane create:")
print(f"  Vento Basso (<8):     {list(vento_basso)}")
print(f"  Vento Medio (8-14):   {list(vento_medio)}")
print(f"  Vento Alto (>=15):    {list(vento_alto)}")

# --- MICRO-CHECKPOINT ---
# Ogni riga deve appartenere a esattamente una categoria
totale = vento_basso.sum() + vento_medio.sum() + vento_alto.sum()
assert totale == len(df), f"Errore: {totale} classificati su {len(df)} righe!"
print(f"\n--- Micro-checkpoint: tutte le {len(df)} righe classificate ---")

,Giorno,Temperatura,Umidita,Pioggia,Vento,umido_alto
0,Lun,18,60,0,5,True
1,Mar,20,55,1,12,False
2,Mer,21,58,0,8,False
3,Gio,19,63,1,20,True
4,Ven,23,50,0,7,False


---

## 3.5 Assegnare Etichette con `.loc[]`

**Perche questo passaggio:** Usiamo `.loc[]` per assegnare valori condizionali. Prima impostiamo un default, poi sovrascriviamo per le categorie specifiche.

**Pattern:**
```python
df["Categoria"] = "Default"
df.loc[condizione, "Categoria"] = "Valore"
```

In [ ]:
# === STEP 2: Assegnare le etichette usando .loc[] ===

# Inizializziamo con il valore di default
df["Vento_Class"] = "Basso"

# Sovrascriviamo per le altre categorie
df.loc[vento_medio, "Vento_Class"] = "Medio"
df.loc[vento_alto, "Vento_Class"] = "Alto"

# --- MICRO-CHECKPOINT ---
print("Distribuzione classi vento:")
print(df["Vento_Class"].value_counts())

# Verifichiamo coerenza: vento alto deve avere Vento >= 15
if df["Vento_Class"].eq("Alto").any():
    min_vento_alto = df.loc[df["Vento_Class"] == "Alto", "Vento"].min()
    assert min_vento_alto >= 15, f"Errore: vento 'Alto' con valore {min_vento_alto}!"
    print(f"\n--- Micro-checkpoint: min vento nella classe 'Alto' = {min_vento_alto} km/h (OK) ---")

df

,Giorno,Temperatura,Umidita,Pioggia,Vento,umido_alto,Vento_Class
0,Lun,18,60,0,5,True,Basso
1,Mar,20,55,1,12,False,Medio
2,Mer,21,58,0,8,False,Medio
3,Gio,19,63,1,20,True,Alto
4,Ven,23,50,0,7,False,Basso
5,Sab,25,45,0,6,False,Basso
6,Dom,22,52,1,15,False,Alto


---

## 3.6 GroupBy Multi-Colonna per Aggregazioni

**Perche questo passaggio:** Vogliamo calcolare la media di Temperatura e Umidita per ogni combinazione di Pioggia e Vento_Class. Questo rivela pattern nascosti nei dati.

**Concetto chiave:** `.groupby(["Col1", "Col2"])` crea gruppi per ogni combinazione unica.

In [ ]:
# === GROUPBY MULTI-COLONNA ===

# Raggruppiamo per Pioggia (0/1) e Vento_Class (Basso/Medio/Alto)
# Calcoliamo la media di Temperatura e Umidita per ogni combinazione
df_grouped = df.groupby(["Pioggia", "Vento_Class"])[["Temperatura", "Umidita"]].mean()

print("Media di Temperatura e Umidita per gruppo:")
print(f"Numero di gruppi: {len(df_grouped)}")
print()

# --- MICRO-CHECKPOINT ---
# Il risultato deve avere MultiIndex (Pioggia, Vento_Class)
assert isinstance(df_grouped.index, pd.MultiIndex), "Deve avere MultiIndex!"
print("--- Micro-checkpoint: MultiIndex creato correttamente ---")
print()

df_grouped

Temperatura    Umidita
Pioggia Vento_Class                        
0       Basso               22.0  51.666667
        Medio               21.0  58.000000
1       Alto                20.5  57.500000
        Medio               20.0  55.000000

---

## 3.7 Transform: Aggiungere Statistiche di Gruppo a Ogni Riga

**Perche questo passaggio:** Con `.transform("mean")` aggiungiamo la media del gruppo a OGNI riga, senza ridurre il DataFrame. Questo permette di confrontare ogni valore con la media del suo gruppo.

**Differenza cruciale:**
- `.mean()` restituisce 1 valore per gruppo (riduce le righe)
- `.transform("mean")` restituisce N valori (stesse righe dell'originale)

In [ ]:
# === TRANSFORM: Media di gruppo assegnata a ogni riga ===

# Per ogni riga, calcoliamo la media dell'umidita del suo gruppo (Pioggia x Vento_Class)
df["Umidita_Media_Gruppo"] = df.groupby(["Pioggia", "Vento_Class"])["Umidita"].transform("mean")

# --- MICRO-CHECKPOINT ---
# Verifichiamo che la lunghezza sia invariata
assert len(df["Umidita_Media_Gruppo"]) == len(df), "Transform deve mantenere la lunghezza!"
print(f"Colonna 'Umidita_Media_Gruppo' creata con {len(df)} valori (stesso numero di righe)")

# Calcoliamo quanto ogni giorno e sopra/sotto la media del suo gruppo
df["Diff_da_Media"] = df["Umidita"] - df["Umidita_Media_Gruppo"]

print("\nInterpretazione Diff_da_Media:")
print("  > 0: umidita sopra la media del gruppo")
print("  < 0: umidita sotto la media del gruppo")
print("  = 0: esattamente nella media")

df

,Giorno,Temperatura,Umidita,Pioggia,Vento,umido_alto,Vento_Class,Umidita_Vento
0,Lun,18,60,0,5,True,Basso,51.666667
1,Mar,20,55,1,12,False,Medio,55.000000
2,Mer,21,58,0,8,False,Medio,58.000000
3,Gio,19,63,1,20,True,Alto,57.500000
4,Ven,23,50,0,7,False,Basso,51.666667
5,Sab,25,45,0,6,False,Basso,51.666667
6,Dom,22,52,1,15,False,Alto,57.500000


---

# SEZIONE 4 — Metodi Spiegati

## Statistiche Descrittive

| Metodo | Sintassi | Output |
|--------|----------|--------|
| `.quantile(q)` | `df["Col"].quantile(0.75)` | Valore al percentile q |
| `.mean()` | `df["Col"].mean()` | Media aritmetica |
| `.median()` | `df["Col"].median()` | Mediana (50° percentile) |
| `.std()` | `df["Col"].std()` | Deviazione standard |

## Creazione Colonne

| Pattern | Esempio | Risultato |
|---------|---------|-----------|
| Booleana | `df["Flag"] = df["A"] > soglia` | True/False |
| Con `.loc[]` | `df.loc[mask, "Cat"] = "Val"` | Assegnazione condizionale |
| Aritmetica | `df["C"] = df["A"] - df["B"]` | Colonna calcolata |

## GroupBy e Aggregazioni

| Metodo | Sintassi | Output |
|--------|----------|--------|
| `.groupby()` | `df.groupby("Col")` | Oggetto GroupBy |
| `.mean()` | `df.groupby("A")["B"].mean()` | Media per gruppo (ridotto) |
| `.sum()` | `df.groupby("A")["B"].sum()` | Somma per gruppo |
| `.count()` | `df.groupby("A")["B"].count()` | Conteggio per gruppo |
| `.agg()` | `df.groupby("A").agg({"B": "mean", "C": "sum"})` | Aggregazioni multiple |

## Transform

| Metodo | Sintassi | Output |
|--------|----------|--------|
| `.transform("mean")` | `df.groupby("A")["B"].transform("mean")` | Media gruppo per ogni riga |
| `.transform("sum")` | `df.groupby("A")["B"].transform("sum")` | Somma gruppo per ogni riga |
| `.transform(func)` | `df.groupby("A")["B"].transform(lambda x: x - x.mean())` | Funzione custom |

## GroupBy Multi-Colonna

| Sintassi | Cosa Fa |
|----------|---------|
| `df.groupby(["A", "B"])` | Gruppi per combinazioni di A e B |
| `df.groupby(["A", "B"])["C"].mean()` | Media di C per ogni combinazione |
| Risultato | MultiIndex con (A, B) come indice |

---

# SEZIONE 5 — Glossario

| Termine | Definizione |
|---------|-------------|
| **Quantile** | Valore che divide i dati ordinati in parti proporzionali |
| **Percentile** | Quantile espresso in percentuale (es. 75° percentile = Q3) |
| **Q1, Q2, Q3** | Primo, secondo (mediana), terzo quartile |
| **IQR** | Interquartile Range = Q3 - Q1, misura di dispersione |
| **Categorizzazione** | Trasformazione di variabile continua in classi discrete |
| **GroupBy** | Operazione che raggruppa righe per valori di una/piu colonne |
| **Split-Apply-Combine** | Paradigma: dividi per gruppi, applica funzione, combina risultati |
| **Aggregazione** | Riduzione di piu valori a uno solo (media, somma, conteggio) |
| **Transform** | Applicazione di funzione che mantiene la lunghezza originale |
| **MultiIndex** | Indice gerarchico con piu livelli (risultato di groupby multi-colonna) |
| **Flag** | Colonna booleana usata per classificazione binaria |
| **Soglia Dinamica** | Soglia calcolata dai dati (es. media, quantile) vs. fissa |
| **Feature Derivata** | Nuova colonna creata da trasformazioni di colonne esistenti |

---

# SEZIONE 6 — Errori Comuni e Debug Rapido

## Errore 1: Confondere `.mean()` con `.transform("mean")`

```python
# SBAGLIATO - riduce a un valore per gruppo
df["Media"] = df.groupby("Cat")["Val"].mean()  # ValueError: lunghezza diversa!

# CORRETTO - mantiene la lunghezza
df["Media"] = df.groupby("Cat")["Val"].transform("mean")
```

## Errore 2: Dimenticare le Parentesi in Condizioni Multiple

```python
# SBAGLIATO
vento_medio = df["Vento"] >= 8 & df["Vento"] < 15  # Errore precedenza!

# CORRETTO
vento_medio = (df["Vento"] >= 8) & (df["Vento"] < 15)
```

## Errore 3: Usare `.loc[]` Senza Colonna

```python
# SBAGLIATO - assegna a TUTTE le colonne!
df.loc[mask] = "Valore"

# CORRETTO - specifica la colonna
df.loc[mask, "Colonna"] = "Valore"
```

## Errore 4: Non Inizializzare la Colonna Prima di `.loc[]`

```python
# SBAGLIATO - colonna non esiste ancora
df.loc[cond1, "Cat"] = "A"
df.loc[cond2, "Cat"] = "B"  # Le righe non in cond1 o cond2 saranno NaN!

# CORRETTO - inizializza con default
df["Cat"] = "Default"
df.loc[cond1, "Cat"] = "A"
df.loc[cond2, "Cat"] = "B"
```

## Errore 5: Quantile Fuori Range [0, 1]

```python
# SBAGLIATO - percentile come intero
df["Col"].quantile(75)  # Errore o risultato errato!

# CORRETTO - decimale tra 0 e 1
df["Col"].quantile(0.75)
```

## Errore 6: GroupBy su Colonna Inesistente

```python
# SBAGLIATO - typo nel nome
df.groupby("Categria")["Val"].mean()  # KeyError!

# CORRETTO - verifica i nomi prima
print(df.columns)
df.groupby("Categoria")["Val"].mean()
```

## Errore 7: Aspettarsi Modifica In-Place

```python
# ATTENZIONE - groupby().mean() restituisce NUOVO oggetto
df.groupby("Cat")["Val"].mean()  # df NON cambia!

# Per usare il risultato, salvalo
result = df.groupby("Cat")["Val"].mean()
```

---

# SEZIONE 7 — Conclusione Operativa

## Cosa Hai Imparato

In questa lezione hai acquisito competenze avanzate di analisi dati:

1. **Quantili come soglie:** definire "alto/basso" basandosi sui dati
2. **Colonne booleane (flag):** classificazione binaria rapida
3. **Categorizzazione:** trasformare numeri in classi significative
4. **GroupBy multi-colonna:** analizzare combinazioni di fattori
5. **Transform:** aggiungere statistiche di gruppo a ogni riga
6. **Confronti intra-gruppo:** quanto sopra/sotto la media del gruppo

## Pattern Ricorrente: Analisi per Gruppi

```python
# 1. Categorizza variabili continue
df["Cat"] = "Default"
df.loc[condizione, "Cat"] = "Valore"

# 2. Raggruppa e calcola statistiche
df.groupby("Cat")["Valore"].mean()

# 3. Aggiungi statistica a ogni riga
df["Media_Gruppo"] = df.groupby("Cat")["Valore"].transform("mean")

# 4. Confronta con la media del gruppo
df["Sopra_Media"] = df["Valore"] > df["Media_Gruppo"]
```

## Collegamento alla Prossima Lezione

Nella Lezione 5 approfondiremo:
- Introduzione a Scikit-learn
- Train-test split
- Primo modello di Machine Learning

---

# SEZIONE 8 — Checklist di Fine Lezione

Prima di procedere alla prossima lezione, verifica di saper fare tutto quanto segue:

- [ ] So calcolare quantili con `.quantile(q)` dove q e tra 0 e 1
- [ ] Capisco la differenza tra Q1, Q2 (mediana), Q3
- [ ] So creare colonne booleane (flag) basate su condizioni
- [ ] So categorizzare variabili continue usando maschere e `.loc[]`
- [ ] So usare `.groupby()` per raggruppare dati
- [ ] So applicare `.groupby()` su piu colonne contemporaneamente
- [ ] Capisco il paradigma Split-Apply-Combine
- [ ] So la differenza tra `.mean()` (riduce) e `.transform("mean")` (mantiene righe)
- [ ] So usare `.transform()` per aggiungere statistiche di gruppo a ogni riga
- [ ] So calcolare quanto un valore e sopra/sotto la media del suo gruppo
- [ ] Ricordo di inizializzare le colonne prima di usare `.loc[]` per assegnazioni
- [ ] So che `.quantile(0.75)` e diverso da `.quantile(75)`

**Se hai dubbi su qualche punto, rileggi la sezione corrispondente prima di proseguire.**

---

# SEZIONE 9 — Changelog Didattico

| Versione | Data       | Modifiche                                                                 |
|----------|------------|---------------------------------------------------------------------------|
| 1.0      | Originale  | Versione iniziale del notebook                                           |
| 2.0      | 2025-12-30 | Ristrutturazione completa secondo template standard a 9 sezioni          |
|          |            | + Nuovo header con obiettivi, prerequisiti e indice                      |
|          |            | + Aggiunta SEZIONE 1: Teoria concettuale (quantili, groupby, transform)  |
|          |            | + Aggiunta SEZIONE 2: Schema mentale / mappa decisionale                 |
|          |            | + Riorganizzata SEZIONE 3: Notebook dimostrativo con spiegazioni         |
|          |            | + Aggiunti "Perche questo passaggio" prima di ogni cella di codice       |
|          |            | + Aggiunti micro-checkpoint con asserzioni e sanity check                |
|          |            | + Aggiunta colonna Diff_da_Media per confronto intra-gruppo              |
|          |            | + Migliorati commenti inline in ogni cella di codice                     |
|          |            | + Aggiunta SEZIONE 4: Metodi spiegati con tabelle di riferimento         |
|          |            | + Aggiunta SEZIONE 5: Glossario (13 termini)                             |
|          |            | + Aggiunta SEZIONE 6: Errori comuni e debug rapido (7 errori)            |
|          |            | + Aggiunta SEZIONE 7: Conclusione operativa                              |
|          |            | + Aggiunta SEZIONE 8: Checklist di fine lezione (12 items)               |
|          |            | + Aggiunta SEZIONE 9: Changelog didattico                                |

---

**Fine della Lezione 4**